# Generator code

In [ ]:
from typing import List


# def fib(n: int) -> List[int]:
#     numbers = []
#     current, nxt = 0, 1
#     while len(numbers) < n:
#         current, nxt = nxt, current + nxt
#         numbers.append(current)
#
#     return numbers


def fib():
    current, nxt = 0, 1
    while True:
        current, nxt = nxt, current + nxt
        yield current


result = fib()

for n in result:
    print(n, end=', ')
    if n > 10000:
        break

print()
print("Done")

In [ ]:
# Starter code
import datetime
import colorama
import random
import time


def main():
    t0 = datetime.datetime.now()
    print(colorama.Fore.WHITE + "App started.", flush=True)
    data = []
    generate_data(10, data)
    generate_data(10, data)
    process_data(20, data)

    dt = datetime.datetime.now() - t0
    print(colorama.Fore.WHITE + "App exiting, total time: {:,.2f} sec.".format(dt.total_seconds()), flush=True)


def generate_data(num: int, data: list):
    for idx in range(1, num + 1):
        item = idx*idx
        data.append((item, datetime.datetime.now()))

        print(colorama.Fore.YELLOW + " -- generated item {}".format(idx), flush=True)
        time.sleep(random.random() + .5)


def process_data(num: int, data: list):
    processed = 0
    while processed < num:
        item = data.pop(0)
        if not item:
            time.sleep(.01)
            continue

        processed += 1
        value = item[0]
        t = item[1]
        dt = datetime.datetime.now() - t

        print(colorama.Fore.CYAN +
              " +++ Processed value {} after {:,.2f} sec.".format(value, dt.total_seconds()), flush=True)
        time.sleep(.5)


if __name__ == '__main__':
    main()

In [2]:
# making it asynch
import datetime
import colorama
import random
import time

import asyncio

import nest_asyncio
nest_asyncio.apply()

def main():
    
    loop = asyncio.get_event_loop()
    
    t0 = datetime.datetime.now()
    print(colorama.Fore.WHITE + "App started.", flush=True)
    
#     data = []
    data = asyncio.Queue()
    
    task = asyncio.gather(
    generate_data(10, data),
    generate_data(10, data),
    process_data(5, data),
    process_data(5, data),
    process_data(5, data),
    process_data(5, data)      
    )
    
#     task = asyncio.gather(
#     generate_data(10, data),
#     generate_data(10, data),
#     process_data(5, data),
#     process_data(5, data),
#     process_data(5, data),
#     process_data(5, data)
#     )
    
    loop.run_until_complete(task)
        
    dt = datetime.datetime.now() - t0
    print(colorama.Fore.WHITE + "App exiting, total time: {:,.2f} sec.".format(dt.total_seconds()), flush=True)

async def generate_data(num: int, data: asyncio.Queue):
    for idx in range(1, num + 1):
        item = idx*idx
        work = (item, datetime.datetime.now())
        await data.put(work)
#         data.append((item, datetime.datetime.now()))

        print(colorama.Fore.YELLOW + " -- generated item {}".format(idx), flush=True)
#         time.sleep(random.random() + .5)
        await asyncio.sleep(random.random() + .5)


async def process_data(num: int, data: asyncio.Queue):
    processed = 0
    while processed < num:
#         item = data.pop(0)
#         if not item:
#             time.sleep(.01)
#             continue
        item = await data.get()

        processed += 1
        value = item[0]
        t = item[1]
        dt = datetime.datetime.now() - t

        print(colorama.Fore.CYAN +
              " +++ Processed value {} after {:,.2f} sec.".format(value, dt.total_seconds()), flush=True)
#         time.sleep(.5)
        await asyncio.sleep(.5)

if __name__ == '__main__':
    main()

App started.
 -- generated item 1
 -- generated item 1
 +++ Processed value 1 after 0.00 sec.
 +++ Processed value 1 after 0.00 sec.
 -- generated item 2
 +++ Processed value 4 after 0.00 sec.
 -- generated item 2
 +++ Processed value 4 after 0.00 sec.
 -- generated item 3
 +++ Processed value 9 after 0.01 sec.
 -- generated item 3
 +++ Processed value 9 after 0.00 sec.
 -- generated item 4
 +++ Processed value 16 after 0.00 sec.
 -- generated item 4
 +++ Processed value 16 after 0.00 sec.
 -- generated item 5
 +++ Processed value 25 after 0.01 sec.
 -- generated item 5
 +++ Processed value 25 after 0.01 sec.
 -- generated item 6
 +++ Processed value 36 after 0.00 sec.
 -- generated item 6
 +++ Processed value 36 after 0.00 sec.
 -- generated item 7
 +++ Processed value 49 after 0.00 sec.
 -- generated item 7
 +++ Processed value 49 after 0.01 sec.
 -- generated item 8
 +++ Processed value 64 after 0.00 sec.
 -- generated item 8
 +++ Processed value 64 after 0.01 sec.
 -- generated ite

In [6]:
# web scrape
import requests
import bs4
from colorama import Fore

import asyncio
import aiohttp

import nest_asyncio
nest_asyncio.apply()


def main():
#     get_title_range()
    # create loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(get_title_range(loop))
    print("Done.")


async def get_html(episode_number: int) -> str:
    print(Fore.YELLOW + f"Getting HTML for episode {episode_number}", flush=True)

    url = f'https://talkpython.fm/{episode_number}'
#     resp = await requests.get(url)
#     resp.raise_for_status()

    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            resp.raise_for_status()

            return await resp.text()


def get_title(html: str, episode_number: int) -> str:
    print(Fore.CYAN + f"Getting TITLE for episode {episode_number}", flush=True)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    header = soup.select_one('h1')
    if not header:
        return "MISSING"

    return header.text.strip()


async def get_title_range(loop):
    # Please keep this range pretty small to not DDoS my site. ;)
    
    tasks = []
    for n in range(185, 200):
        tasks.append((loop.create_task(get_html(n)), n))
#         html = await get_html(n)

    for task, n in tasks:
        html = await task
        title = get_title(html, n)
        print(Fore.WHITE + f"Title found: {title}", flush=True)


if __name__ == '__main__':
    main()

Getting HTML for episode 185
Getting HTML for episode 186
Getting HTML for episode 187
Getting HTML for episode 188
Getting HTML for episode 189
Getting HTML for episode 190
Getting HTML for episode 191
Getting HTML for episode 192
Getting HTML for episode 193
Getting HTML for episode 194
Getting HTML for episode 195
Getting HTML for episode 196
Getting HTML for episode 197
Getting HTML for episode 198
Getting HTML for episode 199
Getting TITLE for episode 185
Title found: Episode #185: Creating a Python 3 Culture at Facebook
Getting TITLE for episode 186
Title found: Episode #186: 100 Days of Python in a Magical Universe
Getting TITLE for episode 187
Title found: Episode #187: Secure all the things with HubbleStack
Getting TITLE for episode 188
Title found: Episode #188: Async for the Pythonic web with Sanic
Getting TITLE for episode 189
Title found: Episode #189: War Stories of the Developer Evangelists
Getting TITLE for episode 190
Title found: Episode #190: Teaching Django
Getting 